<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [21]</a>'.</span>

# Filters and combines existing clusters based on tools used, start and end dates, and organization
## Rebuilds all clusters (will need to run daily)

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'ContactToolAssociation__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"purdue99272844vp8Z6qq0teuPQmirGaaPAt4",   
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [2]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

## Pull data from SF - this will be a post SF filter

In [3]:
##########################################################################################
## 1 - Pull all tool_usage_cluster__c information
##     i) ID
##     ii) Ending_Date__c
##     iii) Starting_Date__c
##     iv) Organization__c
##     v) Semester__c
## 2 - Pull all ContactToolClusterAssociation__c information
##     i) Tool_Usage_Cluster__c
##     ii) Contact__c
## 3 - Pull all ToolToolClusterAssociation__c information
##     i) Tool_Usage_Cluster__c
##     ii) Tool__c
## 4 - determine which tool_usage_cluster__c are suspicious 
##     i) first, group tool_usage_cluster__c by tool__c used
##     ii) check starting and ending dates - if they overlap, make a note of the tool_usage_clusters
## 5 - combining tool_usage_cluster__c 
##     i) use one of the existing Ids, mark the others for deletion
##     ii) when combining, pull the relevant Tool__c information and Contact__c information
##
##########################################################################################

In [4]:
db_1 = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [5]:
tool_cluster_df = db_1.query_data('Select ID, Ending_Date__c, Starting_Date__c, Organization__c,Semester__c \
                        from tool_usage_cluster__c')

[Success] Bulk job creation successful. Job ID = 7505w00000TmOtWAAV


{"id":"7505w00000TmOtWAAV","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T17:11:43.000+0000","systemModstamp":"2021-01-21T17:11:43.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000TmOtWAAV","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T17:11:43.000+0000","systemModstamp":"2021-01-21T17:11:47.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":3944,"retries":0,"totalProcessingTime":696}
[Success] Bulk job completed successfully.


In [6]:
display(tool_cluster_df.head(5))

,Ending_Date__c,Id,Organization__c,Semester__c,Starting_Date__c
0,2011-01-30,a0w5w000009Q7CuAAK,NaN,Fall,2010-09-05
1,2018-11-30,a0w5w000009Q7CvAAK,a0r5w00000V42iyAAB,Fall,2018-11-29
2,2019-11-28,a0w5w000009Q7CwAAK,a0r5w00000V42iyAAB,Fall,2019-11-04
3,2017-10-29,a0w5w000009Q7CxAAK,a0r5w00000V42cCAAR,Fall,2017-10-21
4,2018-01-21,a0w5w000009Q7CyAAK,a0r5w00000V42cCAAR,Spring,2018-01-15


In [7]:
contact_in_cluster_df = db_1.query_data('Select Id, Name, Tool_Usage_Cluster__c, Contact__c from ContactToolClusterAssociation__c')

[Success] Bulk job creation successful. Job ID = 7505w00000TmOtqAAF


{"id":"7505w00000TmOtqAAF","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T17:11:56.000+0000","systemModstamp":"2021-01-21T17:11:56.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000TmOtqAAF","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T17:11:56.000+0000","systemModstamp":"2021-01-21T17:11:58.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":162}


{"id":"7505w00000TmOtqAAF","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T17:11:56.000+0000","systemModstamp":"2021-01-21T17:12:07.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":91675,"retries":0,"totalProcessingTime":7910}
[Success] Bulk job completed successfully.


In [8]:
display(contact_in_cluster_df.head(5))

,Contact__c,Id,Name,Tool_Usage_Cluster__c
0,0035w000034I3dJAAS,a0x5w00000bwbMzAAI,blc070_988,a0w5w000009Q7ufAAC
1,0035w000034I3giAAC,a0x5w00000bwbN0AAI,michaeljh_988,a0w5w000009Q7ufAAC
2,0035w000034I3fOAAS,a0x5w00000bwbN1AAI,acadiak_988,a0w5w000009Q7ufAAC
3,0035w000031VzX2AAK,a0x5w00000bwbN2AAI,jbascom24_988,a0w5w000009Q7ufAAC
4,0035w000034I3ffAAC,a0x5w00000bwbN3AAI,davidwilliams2016_988,a0w5w000009Q7ufAAC


In [9]:
tools_in_cluster_df = db_1.query_data('Select Id, Name, Tool_Usage_Cluster__c, Tool__c from ToolToolClusterAssociation__c')

[Success] Bulk job creation successful. Job ID = 7505w00000TmOuPAAV


{"id":"7505w00000TmOuPAAV","operation":"query","object":"ToolToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T17:12:21.000+0000","systemModstamp":"2021-01-21T17:12:21.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000TmOuPAAV","operation":"query","object":"ToolToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T17:12:21.000+0000","systemModstamp":"2021-01-21T17:12:27.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":6002,"retries":0,"totalProcessingTime":578}
[Success] Bulk job completed successfully.


In [10]:
display(tools_in_cluster_df.head(5))

,Id,Name,Tool_Usage_Cluster__c,Tool__c
0,a0y5w00000MzuVXAAZ,nanocmos_103,a0w5w000009Q7EVAA0,a0s5w00000k5MQJAA2
1,a0y5w00000MzvUQAAZ,geneticalgo_1,a0w5w00000A8OFeAAN,a0s5w00000k5MXTAA2
2,a0y5w00000MzvURAAZ,geneticalgo_2,a0w5w00000A8OFfAAN,a0s5w00000k5MXTAA2
3,a0y5w00000MzvUSAAZ,geneticalgo_3,a0w5w00000A8OFgAAN,a0s5w00000k5MXTAA2
4,a0y5w00000MzvUTAAZ,bmcsuite_4,a0w5w000009Q7CuAAK,a0s5w00000k5MRtAAM


### determining suspicious tool_usage_clusters__c

In [11]:
## overarching goal: determine the cluster IDs that need further investigation

# grouping the clusters by tools used (i.e., clusters by tool)
tools_dict = {j: [] for j in tools_in_cluster_df['Tool__c'].to_list()} #stores relevant indexes per org

for i,j in enumerate(tools_in_cluster_df['Tool__c'].to_list()):
    tools_dict[j].append(i)

unique_tools = list(tools_dict.keys())

In [12]:
from copy import deepcopy

In [13]:
## check start and end dates for overlaps
overlaps = ['init'] # del index 0 later

cluster_ids = tools_in_cluster_df['Tool_Usage_Cluster__c'].to_list()
tool_cluster_id_indexes = tool_cluster_df['Id'].to_list()

for i,ii in enumerate(unique_tools):
    for j in tools_dict[ii]: # this requires a sequential comparison
        ## tools dict contains the tools_in_cluster_df index
        temp_set = deepcopy(tools_dict[ii])
        del temp_set[temp_set.index(j)]
        
        # pull id
        current_index = tool_cluster_id_indexes.index(cluster_ids[j]) #integer index
        
        current_org = tool_cluster_df.iloc[current_index,2]
        current_sem = tool_cluster_df.iloc[current_index,3]
        current_start = datetime.datetime.strptime(tool_cluster_df.iloc[current_index,-1],'%Y-%m-%d')
        current_stop = datetime.datetime.strptime(tool_cluster_df.iloc[current_index,0],'%Y-%m-%d')
        
        # sequential compare
        for k in temp_set:
            temp_index = tool_cluster_id_indexes.index(cluster_ids[k]) #integer index
            
            temp_org = tool_cluster_df.iloc[temp_index,2]
            temp_sem = tool_cluster_df.iloc[temp_index,3]
            temp_start = datetime.datetime.strptime(tool_cluster_df.iloc[temp_index,-1],'%Y-%m-%d')
            temp_stop = datetime.datetime.strptime(tool_cluster_df.iloc[temp_index,0],'%Y-%m-%d')
        
            if temp_org == current_org and temp_sem == current_sem:
                # now check dates for overlap - if there is overlap, assume same cluster
                if temp_start > current_stop or temp_stop < current_start: #case 1 - () [], # case 2 - [] ()
                    # do nothing
                    filler = 0
                else:
                    # include the overlap as a tuple
                    overlaps.append([cluster_ids[j],cluster_ids[k]])


In [14]:
## filter the set of overlaps
if overlaps[0] == 'init':
    del overlaps[0]

for i,j in enumerate(overlaps):
    j.sort()
    overlaps[i] = j

overlaps.sort()

In [15]:
overlaps[:5]

[]

In [16]:
import itertools

In [17]:
overlaps2 = list(k for k,_ in itertools.groupby(overlaps))

In [18]:
overlaps2[:9]

[]

In [19]:
print(len(overlaps),len(overlaps2))

0 0


#### now that we have the set of overlapping clusters, we need to specify a main one

In [20]:
## main cluster characteristics
## Part A
## 1. combine overlaps together (i.e., three overlapping clusters combined into 1) 
## 2. decide the main cluster
## Part B
## 1. start and end dates realignment - i.e., should take the min of all the start dates and max of all the ends
## 2. combine contacts
## 3. create two DataFrames, one for upsert and the other for deletion

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [21]:
## Part A
# collapse overlaps2
overlaps3 = ['init']
main_clusters = []

running_temp_init = True

for i,j in enumerate(overlaps2):
    if running_temp_init == True:
        main_clusters.append(j[0])
        overlaps_holder = deepcopy(j)
        running_temp_init = False
    else:
        if j[0] in overlaps_holder and j[1] not in overlaps_holder:
            overlaps_holder.append(j[1])
        else: 
            if j[0] not in overlaps_holder and j[1] not in overlaps_holder: #neither is in overlaps_holder
                overlaps3.append(overlaps_holder)

                # recalc
                main_clusters.append(j[0])
                overlaps_holder = deepcopy(j)

# final iteration
overlaps3.append(overlaps_holder)

NameError: name 'overlaps_holder' is not defined

In [ ]:
len(main_clusters)

In [ ]:
if overlaps3[0] == 'init':
    del overlaps3[0]
    
print(len(overlaps3))

In [ ]:
display(main_clusters[:5])
display(overlaps3[:5])

### Major step - building upsert and deletion dataframes
### Build 3 upserts and 3 deletions [corresponds to tool_usage_cluster__c, ContactToolClusterAssociation__c, ToolToolClusterAssociation__c]

In [ ]:
## Part B
### build the upsert dataframe first
upsert_df = pd.DataFrame(columns=['Id','Ending_Date__c','Starting_Date__c']) #Id of the main cluster
upsert_df['Id'] = deepcopy(main_clusters)

In [ ]:
for i,ii in enumerate(main_clusters):
    main_c_index = tool_cluster_df['Id'].to_list()
    c_start_date = tool_cluster_df.iloc[main_c_index.index(ii),-1]    
    c_end_date = tool_cluster_df.iloc[main_c_index.index(ii),0]
        
    for j,jj in enumerate(overlaps3[i]):
        j_start_date = tool_cluster_df.iloc[main_c_index.index(jj),-1]
        j_end_date = tool_cluster_df.iloc[main_c_index.index(jj),0]
        
        if j_start_date < c_start_date:
            c_start_date = deepcopy(j_start_date)
        
        if j_end_date > c_end_date:
            c_end_date = deepcopy(j_end_date)
        
    upsert_df.iloc[i,1:] = [c_end_date,c_start_date]

In [ ]:
display(upsert_df.head(5))
display(upsert_df.tail(5))

In [ ]:
### buliding the delete dataframe - should be all Ids in overlaps that are not main clusters
deletion_df = pd.DataFrame(columns=['Id']) #Id of the main cluster

deletion_ids = []

for i,ii in enumerate(main_clusters):
    for j in overlaps3[i][1:]:
        deletion_ids.append(j)
deletion_df['Id'] = deletion_ids

In [ ]:
display(deletion_df.head(3))

In [ ]:
## summary up to this point
## upsert_df db.send_data(upsert_df)
## db.object_id = 'tool_usage_cluster__c'
## db.external_id = 'Id'

## deletion_df db.delete_data(deletion_df)
## db.object_id = 'tool_usage_cluster__c'
## db.external_id = 'Id'

In [ ]:
## updating contact and cluster mapping
display(contact_in_cluster_df.head(2))
print(contact_in_cluster_df.shape)

In [ ]:
# NOTE: there are contacts that belong to non-overlapping clusters
contact_usage_clusters = contact_in_cluster_df['Tool_Usage_Cluster__c'].to_list()
contacts = contact_in_cluster_df['Contact__c'].to_list()
change_indexes = []

## check if current cluster belongs to overlaps3 - first need to convert overlaps3 into a direct list
overlaps4 = list(itertools.chain.from_iterable(overlaps3))

In [ ]:
print(len(overlaps4))

In [ ]:
## populate the contacts that need changing
for i,j in enumerate(contact_usage_clusters):
    if j in overlaps4:
        change_indexes.append(i)

In [ ]:
print(len(change_indexes))

In [ ]:
## search for each index that needs changing
for i in change_indexes:
    if contact_usage_clusters[i] not in main_clusters:
        #print('testouter')
        
        # need to find its main_cluster
        found_main = False
        main_counter = 0
        while found_main == False:
            if contact_usage_clusters[i] in overlaps3[main_counter]:
                #print('test')
                found_main = True
                contact_usage_clusters[i] = overlaps3[main_counter][0]
            else:
                main_counter += 1

In [ ]:
contact_in_cluster_df['Tool_Usage_Cluster__c'] = contact_usage_clusters

In [ ]:
## remove duplicates from contact_in_cluster_df
contact_in_cluster_df = contact_in_cluster_df.drop_duplicates()
display(contact_in_cluster_df.head(2))
print(contact_in_cluster_df.shape)

In [ ]:
## tools - take max, i.e. set all constituent clusters of a main cluster to the main cluster
## then remove duplicates
display(tools_in_cluster_df.head(2))

In [ ]:
tuc = tools_in_cluster_df['Tool_Usage_Cluster__c'].to_list()

# first find those cluster ids that are clustered
tool_cluster_ids = []
for i,j in enumerate(tuc):
    if j in overlaps4:
        tool_cluster_ids.append(i)

In [ ]:
print(len(tool_cluster_ids))

In [ ]:
for i in tool_cluster_ids:
    if tuc[i] not in main_clusters: #then need to find its corresponding main cluster
        found_main = False
        main_counter = 0
        
        while found_main == False:
            if tuc[i] in overlaps3[main_counter]:
                #print('test')
                found_main = True
                tuc[i] = overlaps3[main_counter][0]
            else:
                main_counter += 1

In [ ]:
tools_in_cluster_df['Tool_Usage_Cluster__c'] = tuc
display(tools_in_cluster_df.head(2))

## Send to SF

In [ ]:
db_upsert_del = DB2SalesforceAPI(sf_login_params)

In [ ]:
db_upsert_del.object_id = 'tool_usage_cluster__c'
db_upsert_del.external_id = 'Id'

In [ ]:
db_upsert_del.send_data(upsert_df)

In [ ]:
db_upsert_del.check_bulk_status()

In [ ]:
from pprint import pprint
pprint(db_upsert_del.check_bulk_failed_results())

#### upload contact_in_cluster_df

In [ ]:
db_contact_upsert = DB2SalesforceAPI(sf_login_params)

In [ ]:
## need to delete first, then upload
contact_in_cluster_df_del = pd.DataFrame(contact_in_cluster_df['Id'])
contact_in_cluster_df_up = contact_in_cluster_df.drop(columns='Id')

In [ ]:
display(contact_in_cluster_df_del.head(2))
display(contact_in_cluster_df_up.head(2))

In [ ]:
db_contact_upsert.object_id = 'ContactToolClusterAssociation__c'
db_contact_upsert.external_id = 'Id'

In [ ]:
db_contact_upsert.delete_data(contact_in_cluster_df_del)

In [ ]:
db_contact_upsert.check_bulk_status()

In [ ]:
pprint(db_contact_upsert.check_bulk_failed_results())

In [ ]:
db_contact_upsert.object_id = 'ContactToolClusterAssociation__c'
db_contact_upsert.external_id = 'Name'

In [ ]:
db_contact_upsert.send_data(contact_in_cluster_df_up)

In [ ]:
db_contact_upsert.check_bulk_status()

In [ ]:
pprint(db_contact_upsert.check_bulk_failed_results())

#### tools_in_cluster_df

In [ ]:
db_tool_upsert = DB2SalesforceAPI(sf_login_params)

In [ ]:
## need to delete first, then upload
db_tool_upsert_del = pd.DataFrame(tools_in_cluster_df['Id'])
db_tool_upsert_up = tools_in_cluster_df.drop(columns='Id')

In [ ]:
display(db_tool_upsert_del.head(2))
display(db_tool_upsert_up.head(2))

In [ ]:
db_tool_upsert.external_id = 'Id'
db_tool_upsert.object_id = 'ToolToolClusterAssociation__c'

In [ ]:
db_tool_upsert.delete_data(db_tool_upsert_del)

In [ ]:
db_tool_upsert.check_bulk_status()

In [ ]:
pprint(db_tool_upsert.check_bulk_failed_results())

In [ ]:
## now upload

In [ ]:
db_tool_upsert.external_id = 'Name'
db_tool_upsert.object_id = 'ToolToolClusterAssociation__c'

In [ ]:
db_tool_upsert.send_data(db_tool_upsert_up)

In [ ]:
db_tool_upsert.check_bulk_status()

In [ ]:
pprint(db_tool_upsert.check_bulk_failed_results())

#### delete clustered tool cluster ids

In [ ]:
db_del = DB2SalesforceAPI(sf_login_params)

In [ ]:
db_del.object_id = 'tool_usage_cluster__c'
db_del.external_id = 'id'

In [ ]:
db_del.delete_data(deletion_df)

In [ ]:
display(deletion_df.head(2))

In [ ]:
db_del.check_bulk_status()

In [ ]:
pprint(db_del.check_bulk_failed_results())

In [ ]:
### eventually, want to update DB2 with this information
